In [50]:
import pandas as pd

In [51]:
data_path = '/mnt/disks/external_mssd/'
submission_path = data_path + 'Submissions/'
training_path = data_path + '/training_set/'

In [52]:
csv = training_path + 'log_0_20180715_000000000000.csv'

In [69]:
def get_ground_truth(csv_path):
    ground_truths = []     
    with open(csv_path,'r') as f:
        df = pd.read_csv(f)
        df = df[['session_id','skip_2','session_position','session_length']].loc[df['session_position']*2 > df['session_length']]
        df = df.reset_index()
        current_index = 0
        while current_index < len(df):
            partial_length = df['session_length'].iloc[current_index]-df['session_position'].iloc[current_index]+1
            session_skips = list(df.loc[current_index:current_index+partial_length-1, 'skip_2'])
            ground_truths.append(session_skips)
            current_index += partial_length 
    return ground_truths

In [70]:
ground_truth = get_ground_truth(csv)

In [71]:
ground_truth[0]

[True, True, True, False, True, True, True, True, True, True]

In [73]:
def generate_random_submission(csv_path):
    predictions = []
    with open(csv_path,'r') as f:
        df = pd.read_csv(f)
        df = df[['session_id','track_id_clean','session_position','session_length']].loc[df['session_position']*2 > df['session_length']]
        df = df.reset_index()
        current_index = 0
        while current_index < len(df):
            partial_length = df['session_length'].iloc[current_index]-df['session_position'].iloc[current_index]+1
            current_preds = np.random.choice([0, 1], size=(partial_length,))
            predictions.append(current_preds)
            current_index += partial_length
    return predictions

In [74]:
submission = generate_random_submission(csv)

In [89]:
print(len(submission))

178350


In [90]:
def evaluate(submission,groundtruth):
    ap_sum = 0.0
    first_pred_acc_sum = 0.0
    counter = 0
    for sub, tru in zip(submission, groundtruth):
        if len(sub) != len(tru):
            raise Exception('Line {} should contain {} predictions, but instead contains '
                            '{}'.format(counter+1,len(tru),len(sub)))
        ap_sum += ave_pre(sub,tru,counter)
        first_pred_acc_sum += sub[0] == tru[0]
        counter+=1
    ap = ap_sum/counter
    first_pred_acc = first_pred_acc_sum/counter
    return ap,first_pred_acc

In [91]:
def ave_pre(submission,groundtruth,counter):
    s = 0.0
    t = 0.0
    c = 1.0
    for x, y in zip(submission, groundtruth):
        if x != 0 and x != 1:
            raise Exception('Invalid prediction in line {}, should be 0 or 1'.format(counter))
        if x==y:
            s += 1.0
            t += s / c
        c += 1
    return t/len(groundtruth)

In [92]:
ap,first_pred_acc = evaluate(submission,ground_truth)

In [93]:
print('average precision: {}'.format(ap))
print('first prediction accuracy: {}'.format(first_pred_acc))

average precision: 0.33429380263364245
first prediction accuracy: 0.4992486683487525
